# Interactive LLM Notebook

this notebook provides the front end for interacting with the context-given LLM. 

In [1]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI



In [4]:
os.environ['OPENAI_API_KEY'] = 
##need to load this first... before we load in the config... need to think about HOW this should work

In [5]:
## Eventually make this something to be loaded 
config = {
    'basedir': '../vectorstore/',
    'retrain_str': False,
    'persist_dir': '../vectorstore/chroma/',
    'datadir': '../../journals/',
    'text_splitter': RecursiveCharacterTextSplitter(
                            chunk_size = 1500,
                            chunk_overlap = 150,
                            separators=['<|endoftext|>',"\n\n", "\n", "\n(.+?)\n","(?<=\. )", " ", ""]
                                            ),
    'embeddings':OpenAIEmbeddings(),
    'allowed_special_kwargs': '<|endoftext|>',
    'llm': ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0), }


In [6]:
from utils.utils import retrieve_context_vectordb, get_conversational_llm

In [12]:
config['retriever'] = retrieve_context_vectordb(config)

loading vectordb


In [13]:
qa = get_conversational_llm(config)

In [17]:
question = 'who wrote the paper about fluid mechanics and deep learning?'
chat_history = []
result = qa({"question": question, "chat_history": chat_history})

In [18]:
result

{'question': 'who wrote the paper about fluid mechanics and deep learning?',
 'chat_history': [],
 'answer': 'The paper titled "A review on deep reinforcement learning for fluid mechanics: An update" was written by J. Viquerat, P. Meliga, A. Larcher, and E. Hachem.',
 'source_documents': [Document(page_content='A review on deep reinforcement learning\nfor fluid mechanics: An update\nCite as: Phys. Fluids 34, 111301 (2022); doi: 10.1063/5.0128446\nSubmitted: 28 September 2022 .Accepted: 1 November 2022 .\nPublished Online: 17 November 2022\nJ.Viquerat,a)\nP.Meliga,b)A.Larcher,c)\nand E. Hachemd)\nAFFILIATIONS\nMINES Paristech, CEMEF PSL Research University, 06904 Sophia Antipolis, France\na)Author to whom correspondence should be addressed: jonathan.viquerat@mines-paristech.fr\nb)philippe.meliga@mines-paristech.fr\nc)aurelien.larcher@mines-paristech.fr\nd)elie.hachem@mines-paristech.fr\nABSTRACT\nIn the past couple of years, the interest of the ﬂuid mechanics community for deep reinforc